In [1]:
#imports libraries that will be utilized
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import requests
import scipy.stats as sts
import scipy as spy
import scipy.integrate as integrate
import scipy.optimize as opt
import random
import numpy.linalg as lin
import pickle as pkl
import os

In [2]:
#Run to skip all the data creation and optimization
#Be careful!!!!

SCC=36

with open('GenStack0','rb') as f:
    GenStack0 = pkl.load(f)
    
with open('GenStackCO2','rb') as f:
    GenStackCO2 = pkl.load(f)
    
with open('LoadProfile','rb') as f:
    LoadProfile = pkl.load(f)

with open('HrNaturalGasPrices','rb') as f:
    HrNaturalGasPrices = pkl.load(f)
    
with open('PSeries','rb') as f:
    PSeries = pkl.load(f)
    
with open('Exchange','rb') as f:
    Exchange = pkl.load(f)
    
with open('results_uncstr','rb') as f:
    results_uncstr = pkl.load(f)
    
with open('results_uncstrb','rb') as f:
    results_uncstrb = pkl.load(f)

ModLoadProfile=np.array(LoadProfile[:])-np.array(Exchange[:])


In [2]:
#Loads Generators Stack
#FuelTypes = HYDRO, GAS, WIND, NUCLEAR
#HeatRate [BTU/kWh]
#EmmisionRate [lb/MWh]
GenStack0=pd.read_csv('GenStack.csv',sep=",")
GenStack0.columns=["Gen","FuelType","PMax","HeatRate","EmissionRate","Cost"]

GenStackCO2=pd.read_csv('GenStackCO2v0.csv',sep=",")
GenStackCO2.columns=["Gen","FuelType","PMax","HeatRate","EmissionRate","Cost"]

SCC=36

with open('GenStack0','wb') as f:
    pkl.dump(GenStack0, f)

with open('GenStackCO2','wb') as f:
    pkl.dump(GenStackCO2, f)

In [3]:
LoadProfile0=pd.DataFrame(np.zeros((0,5)))
LoadProfile0.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
TempProfile=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01palIntegrated_csv".format(y,m)
            if m>=10:
                folder="{}{}01palIntegrated_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}palIntegrated.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}palIntegrated.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}palIntegrated.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Load",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","TimeZone","Zone","PTID","Load"]
                    TempProfile=np.hstack((TempProfile,np.array(TempFile['Load'])))                
                    LoadProfile0=pd.concat([LoadProfile0,TempFile],ignore_index=True)

In [4]:
LoadProfile=[]
count=0
hrLoad=0
for pos in range (0,len(TempProfile)):
    if pos == len(TempProfile)-1:
        hrLoad=hrLoad+TempProfile[pos]
        LoadProfile.append(hrLoad)
    if count==11:
        LoadProfile.append(hrLoad)
        count=0
        hrLoad=0
    hrLoad=hrLoad+TempProfile[pos]
    count=count+1
    
with open('LoadProfile','wb') as f:
    pkl.dump(LoadProfile, f)

In [5]:
MonthNaturalGasPrices=[]
with open(os.path.join("Data","MonthlyNG_prices_17_18.csv")) as f:
    NaturalGasPricesTemp=pd.read_csv(f,sep=",",header=None)
    
MonthNaturalGasPrices=NaturalGasPricesTemp.iloc[:,0]

In [6]:
HrNaturalGasPrices=[]

yearcount=0
for m in range (1,25):
    if m>12:
        m=m-12
        yearcount=1
    if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
        for d in range (1,(31*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])
    if m==4 or m==6 or m==9 or m==11:
        for d in range (1,(30*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])
    if m==2:
        for d in range (1,(28*24)+1):
            HrNaturalGasPrices.append(MonthNaturalGasPrices[m-1+(yearcount*12)])

with open('HrNaturalGasPrices','wb') as f:
    pkl.dump(HrNaturalGasPrices, f)

In [7]:
PSeries0=pd.DataFrame(np.zeros((0,6)))
PSeries0.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
TempPSeries=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01rtlbmp_zone_csv".format(y,m)
            if m>=10:
                folder="{}{}01rtlbmp_zone_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}rtlbmp_zone.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}rtlbmp_zone.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}rtlbmp_zone.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","Pricing",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","Zone","PTID","LBMP","MarCLosses","MarCCong"]
                    TempPSeries=np.hstack((TempPSeries,np.array(TempFile['LBMP'])))
                    PSeries0=pd.concat([PSeries0,TempFile],ignore_index=True)

In [8]:
PSeries=[]
count=0
TotCosthr=0
posLoad=0
for pos in range (0,len(TempPSeries)):
    if pos == len(TempProfile)-1:
        TotCosthr=TotCosthr+(TempProfile[pos]*TempPSeries[posLoad])
        PSeries.append(TotCosthr)
    if count==15:
        PSeries.append(TotCosthr)
        count=0
        TotCosthr=0
    if count==0 or count==1 or count==2 or count==3 or count==5 or count==6 or count==7 or count==8 or count==9 or count==10 or count==14:
        TotCosthr=TotCosthr+(TempProfile[posLoad]*TempPSeries[pos])
        posLoad=posLoad+1
    count=count+1

for row in range (0,len(PSeries)):
    PSeries[row]=PSeries[row]/LoadProfile[row]
    if PSeries[row]<0:
        PSeries[row]=0
        
with open('PSeries','wb') as f:
    pkl.dump(PSeries, f)

In [9]:
Exchange0=pd.DataFrame(np.zeros((0,6)))
Exchange0.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
TempExchange=[]
for y in range (2017,2019):
    for m in range (1,13):
        if m==1 or m==3 or m==5 or m==7 or m==8 or m==10 or m==12:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,32):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)
        if m==4 or m==6 or m==9 or m==11:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,31):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)
        if m==2:
            if m<10:
                folder="{}0{}01ExternalLimitsFlows_csv".format(y,m)
            if m>=10:
                folder="{}{}01ExternalLimitsFlows_csv".format(y,m)
            for d in range (1,29):
                if d<10:
                    if m<10:
                        file="{}0{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}0{}ExternalLimitsFlows.csv".format(y,m,d)
                if d>=10:
                    if m<10:
                        file="{}0{}{}ExternalLimitsFlows.csv".format(y,m,d)
                    if m>=10:
                        file="{}{}{}ExternalLimitsFlows.csv".format(y,m,d)
                with open(os.path.join("Data","NYISO","ExternalInterfaces",folder,file)) as f:
                    TempFile=pd.read_csv(f,sep=",")
                    TempFile.columns=["TimeStamp","InterfaceName","PTID","Flow","PosLimit","NegLimit"]
                    TempExchange=np.hstack((TempExchange,np.array(TempFile['Flow'])))
                    Exchange0=pd.concat([Exchange0,TempFile],ignore_index=True)

In [10]:
Exchange = pd.DataFrame(Exchange0)
Exchange['TimeStamp']=pd.to_datetime(Exchange['TimeStamp'])
Exchange['PTID']=np.array(Exchange['PTID'])
Exchange.set_index(Exchange['PTID'],inplace=True)
Exchange=Exchange.drop([23330,23326,23319,23320,23314,23315,23312])
Exchange.set_index(Exchange['TimeStamp'],inplace=True)
Exchange=Exchange.groupby([Exchange.index.year, Exchange.index.month, Exchange.index.day, Exchange.index.hour]).sum()
Exchange['Flow']=Exchange['Flow']/12
Exchange=np.array(Exchange['Flow'])
Daylight2017=Exchange[(31*24)+(28*24)+(11*24)+2]
Daylight2018=Exchange[(365*24)+(31*24)+(28*24)+(10*24)+2]

TempHrExchange=[]
for row in range (0,len(Exchange)):
    if row==(31*24)+(28*24)+(11*24)+2:
        TempHrExchange.append(Daylight2017)
    if row==(365*24)+(31*24)+(28*24)+(10*24)+2:
        TempHrExchange.append(Daylight2018)
    TempHrExchange.append(Exchange[row])
    
Exchange=TempHrExchange
Exchange[((31+28+31+30+31+30+31+31+30+31)*24)+(4*24)+2-1]=Exchange[((31+28+31+30+31+30+31+31+30+31)*24)+(4*24)+2-1]/2
Exchange[((365+31+28+31+30+31+30+31+31+30+31)*24)+(3*24)+2-1]=Exchange[((365+31+28+31+30+31+30+31+31+30+31)*24)+(3*24)+2-1]/2

with open('Exchange','wb') as f:
    pkl.dump(Exchange, f)

In [11]:
ModLoadProfile=np.array(LoadProfile[:])-np.array(Exchange[:])

with open('ModLoadProfile','wb') as f:
    pkl.dump(ModLoadProfile, f)

In [3]:
def MarginalGen(load):
    Maxgen=0
    genid=0
    while Maxgen<load and genid<len(GenStack0['PMax']):
        Maxgen=Maxgen+GenStack0['PMax'][genid]
        genid=genid+1
    return genid-1
def MarginalGenCO2(load):
    Maxgen=0
    genid=0
    while Maxgen<load and genid<len(GenStackCO2['PMax']):
        Maxgen=Maxgen+GenStackCO2['PMax'][genid]
        genid=genid+1
    return genid-1

In [4]:
def MarginalCost(marGen,NGPrice,csource):
    if float(GenStack0.HeatRate.iloc[marGen]):
        marCos=((GenStack0.HeatRate.iloc[marGen])/1000000)*1000*(NGPrice+csource)
    else:
        marCos=1e-8
    return marCos
def MarginalCostCO2(marGen,NGPrice, csource,Pass):
    if float(GenStackCO2.HeatRate.iloc[marGen]):
        marCos=(((GenStackCO2.HeatRate.iloc[marGen])/1000000)*1000*(NGPrice+csource))+(GenStackCO2.Cost.iloc[marGen]*Pass)
    else:
        marCos=1e-8
    return marCos

In [5]:
#Initial weighting matrix
W=np.eye(5)
sim=1200

sigma_0=3
mu_0=3
rho_0=1/2

#Generates matrix with random cumulative probability values
np.random.seed(25)  # Set the random number seed so it gives same answers every time
S_0 = sts.uniform.rvs(0, 1, size=(len(PSeries),sim))

#Function that calculates error based on a cumulative probability value
def epsilonma(matrix,sigma):
    epsm=sts.norm.ppf(matrix,loc=0,scale=sigma)
    return epsm

#Function that calculates a simulated CSource's based on errors, rho, mu and an initial CSource
def simC(errors,rho,mu,cSource0):
    simulatedC=np.array(errors)
    simulatedC[0,:]=np.array(rho*cSource0+(1-rho)*mu+errors[0,:])
    
    for l in range (1,len(PSeries)):
        simulatedC[l,:]=np.array(rho*simulatedC[l-1,:]+(1-rho)*mu+errors[l,:])
    
    return simulatedC

#Function that calculates a simulated P(t) based on Load and CSource
def simP(simC,Load):
    simulatedP=np.array(simC)
    LoadProf=np.array(Load)
    
    for row in range (0, len(PSeries)):
        MarGen=MarginalGen(LoadProf[row])
        simulatedP[row,:]=MarginalCost(MarGen,HrNaturalGasPrices[row],simC[row,:])
    
    return simulatedP

#Calculates data moments
m1=np.mean(PSeries)
m2=np.var(np.array(PSeries[:])*np.array(ModLoadProfile[:]))
corrPL, pvalPL = sts.pearsonr(PSeries,ModLoadProfile)
m3=corrPL
modPt=np.zeros((len(PSeries)-1))
modPt1=np.zeros((len(PSeries)-1))

for count in range (0, len(PSeries)-1):
    modPt[count]=PSeries[count]
    modPt1[count]=PSeries[count+1]

corrPt, pvalPt=sts.pearsonr(modPt,modPt1)
m4=corrPt
m5=np.sum(np.array(PSeries[:])*np.array(ModLoadProfile[:]))

#Defines optimization criterion
def crit(params):
    rho,mu,sigma = params

    #Creates objects to store simulated moments
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    sm5=[]
    modPst=np.zeros((len(PSeries)-1,sim))
    modPst1=np.zeros((len(PSeries)-1,sim))
    modPt=np.zeros((len(PSeries)-1))
    modPt1=np.zeros((len(PSeries)-1))
    
    
    cSource0=mu
    
    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    Cst=np.array(simC(errors,rho,mu,cSource0))
    Pst=np.array(simP(Cst,ModLoadProfile))

    
    #Calculates simulated moments
    for i in range (0, sim):
    
        sm1.append(np.mean(Pst[:,i]))
        sm2.append(np.var(Pst[:,i]*np.array(ModLoadProfile[:])))

        corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(ModLoadProfile[:]))
        sm3.append(corrm3)
        
        for count in range (0, len(PSeries)-1):
            modPst[count,i]=Pst[count,i]
            modPst1[count,i]=Pst[count+1,i]
        
        corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
        sm4.append(corrm4)
        sm5.append(np.sum(np.array(Pst[:,i])*np.array(ModLoadProfile[:])))
        

        
    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4
    err5=(np.mean(sm5)-m5)/m5

    #Creates array of errors
    err=np.array([err1, err2, err3, err4, err5])
    
    #Calculates estimation criterion
    estimator=err.T @ W @ err
    
    return estimator

#Performs optimization starting with initial parameters
params_0=np.array([rho_0,mu_0, sigma_0])
#results_uncstr = opt.minimize(crit, params_0, method='L-BFGS-B',bounds=((-0.99, 0.99),(0.01,20),(0.01,10)),tol=1e-8)
rho, mu, sigma = results_uncstr.x

#Prints estimated parameters
print("Estimated Rho = ", rho)
print("Estimated Mu = ", mu)
print("Estimated Sigma = ", sigma)

#Calculates final values of moments with optimized parameters (repeats the process described in
#   "optimization criteria")
sm1=[]
sm2=[]
sm3=[]
sm4=[]
sm5=[]
modPst=np.zeros((len(PSeries)-1,sim))
modPst1=np.zeros((len(PSeries)-1,sim))
modPt=np.zeros((len(PSeries)-1))
modPt1=np.zeros((len(PSeries)-1))


cSource0=mu

#Calculates variables as per problem set's equations
errors=np.array(epsilonma(S_0, sigma))
Cst=np.array(simC(errors,rho,mu,cSource0))
Pst=np.array(simP(Cst,ModLoadProfile))


#Calculates simulated moments
for i in range (0, sim):

    sm1.append(np.mean(Pst[:,i]))
    sm2.append(np.var(Pst[:,i]*np.array(ModLoadProfile[:])))

    corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(ModLoadProfile[:]))
    sm3.append(corrm3)

    for count in range (0, len(PSeries)-1):
        modPst[count,i]=Pst[count,i]
        modPst1[count,i]=Pst[count+1,i]

    corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
    sm4.append(corrm4)
    sm5.append(np.sum(np.array(Pst[:,i])*np.array(ModLoadProfile[:])))


#Calculates errors (Difference between simulated moments and data moments)
err1=(np.mean(sm1)-m1)/m1
err2=(np.mean(sm2)-m2)/m2
err3=(np.mean(sm3)-m3)/m3
err4=(np.mean(sm4)-m4)/m4
err5=(np.mean(sm5)-m5)/m5

#Creates array of errors
err=np.array([err1, err2, err3, err4, err5])

#Prints final vector of moment differences
print("Vector of differences = ", err)

#Prints value of final estimator
print("Criterion function value =",results_uncstr.fun)

#Calculates errors matrix
#1. Defines function for calculating errors
def s_errors(params):
    
    rho, mu, sigma = params
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    sm5=[]
    modPst=np.zeros((len(PSeries)-1,sim))
    modPst1=np.zeros((len(PSeries)-1,sim))
    modPt=np.zeros((len(PSeries)-1))
    modPt1=np.zeros((len(PSeries)-1))


    cSource0=mu

    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    Cst=np.array(simC(errors,rho,mu,cSource0))
    Pst=np.array(simP(Cst,ModLoadProfile))


    #Calculates simulated moments
    for i in range (0, sim):

        sm1.append(np.mean(Pst[:,i]))
        sm2.append(np.var(Pst[:,i]*np.array(ModLoadProfile[:])))

        corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(ModLoadProfile[:]))
        sm3.append(corrm3)

        for count in range (0, len(PSeries)-1):
            modPst[count,i]=Pst[count,i]
            modPst1[count,i]=Pst[count+1,i]

        corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
        sm4.append(corrm4)
        sm5.append(np.sum(np.array(Pst[:,i])*np.array(ModLoadProfile[:])))


    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4
    err5=(np.mean(sm5)-m5)/m5

    #Creates array of errors
    err=np.array([err1, err2, err3, err4, err5])
    
    return err

#2. Generates Jacobian Matrix
npar=3
nmom=5
h=1e-5
Ematrix=np.zeros((nmom, npar))


for col in range (0,npar):
    derivparamup=np.array([rho, mu, sigma])
    derivparamup[col]=derivparamup[col]+h
    derivparamdown=np.array([rho, mu, sigma])
    derivparamdown[col]=derivparamdown[col]-h
    deltaerror=s_errors(derivparamup)-s_errors(derivparamdown)
    for row in range (0,nmom):
        Ematrix[row,col]=deltaerror[row]/(2*h)
        
        
        
cov2=(1/sim)*lin.pinv(Ematrix.T@ W @ Ematrix)

StandardErra=[]

#3. Calculates and prints Standard errors. Order: m1, m2, m3, m4
for row in range (0,npar):
    StandardErra.append((cov2[row,row])**1/2)

print("Standard Errors of Parameter Vector = ", StandardErra)

Estimated Rho =  0.6376503158636115
Estimated Mu =  1.2178931719978752
Estimated Sigma =  3.5534478212383065
Vector of differences =  [ 0.14881166 -0.13219568 -0.70622063 -0.00108056  0.08213635]
Criterion function value = 0.5451157304682422
Standard Errors of Parameter Vector =  [0.00023236849034598717, 0.003944712657008988, 0.005526624134765092]


In [6]:
#Performs the same calculation performed above, but includes the step-2 weighting matrix

#Calculates 2-step weighting matrix
nmom=5
Ematrix=np.zeros((nmom,sim))
DataMom=np.array([m1,m2,m3,m4,m5])
SimMom=np.array([sm1,sm2,sm3,sm4,sm5])

for col in range (0,sim):
    for row in range (0,nmom):
        Ematrix[row,col]=(SimMom[row,col]-DataMom[row])/DataMom[row]
        
sig2=(1/sim)*(Ematrix @ Ematrix.T)
W2=lin.pinv(sig2)

#Defines new optimization criterion that uses the new 2-step W matrix
def critb(params):
    rho,mu,sigma = params

    #Creates objects to store simulated moments
    sm1=[]
    sm2=[]
    sm3=[]
    sm4=[]
    sm5=[]
    modPst=np.zeros((len(PSeries)-1,sim))
    modPst1=np.zeros((len(PSeries)-1,sim))
    modPt=np.zeros((len(PSeries)-1))
    modPt1=np.zeros((len(PSeries)-1))
    
    
    cSource0=mu
    
    #Calculates variables as per problem set's equations
    errors=np.array(epsilonma(S_0, sigma))
    Cst=np.array(simC(errors,rho,mu,cSource0))
    Pst=np.array(simP(Cst,ModLoadProfile))

    
    #Calculates simulated moments
    for i in range (0, sim):
    
        sm1.append(np.mean(Pst[:,i]))
        sm2.append(np.var(Pst[:,i]*np.array(ModLoadProfile[:])))

        corrm3, pvalm3 = sts.pearsonr(Pst[:,i],np.array(ModLoadProfile[:]))
        sm3.append(corrm3)
        
        for count in range (0, len(PSeries)-1):
            modPst[count,i]=Pst[count,i]
            modPst1[count,i]=Pst[count+1,i]
        
        corrm4, pvalm4 = sts.pearsonr(modPst[:,i],modPst1[:,i])
        sm4.append(corrm4)
        sm5.append(np.sum(np.array(Pst[:,i])*np.array(ModLoadProfile[:])))
        

        
    #Calculates errors (Difference between simulated moments and data moments)
    err1=(np.mean(sm1)-m1)/m1
    err2=(np.mean(sm2)-m2)/m2
    err3=(np.mean(sm3)-m3)/m3
    err4=(np.mean(sm4)-m4)/m4
    err5=(np.mean(sm5)-m5)/m5

    #Creates array of errors
    err=np.array([err1, err2, err3, err4, err5])
    
    #Calculates estimation criterion
    estimatorb=err.T @ W2 @ err
    
    return estimatorb

#Performs optimization starting with initial parameters
params_a=np.array([rho,mu,sigma])
#results_uncstrb = opt.minimize(critb, params_a, method='L-BFGS-B',bounds=((-0.99, 0.99),(0.01,20),(0.01,10)),tol=1e-8)
rho_b, mu_b, sigma_b = results_uncstrb.x

#Prints estimated parameters
print("Estimated Rho = ", rho_b)
print("Estimated Mu = ", mu_b)
print("Estimated Sigma = ", sigma_b)


#Calculates final values of moments with optimized parameters (repeats the process described in
#   "optimization criteria")
sm1_b=[]
sm2_b=[]
sm3_b=[]
sm4_b=[]
sm5_b=[]
modPst_b=np.zeros((len(PSeries)-1,sim))
modPst1_b=np.zeros((len(PSeries)-1,sim))
modPt_b=np.zeros((len(PSeries)-1))
modPt1_b=np.zeros((len(PSeries)-1))


cSource0_b=mu_b

#Calculates variables as per problem set's equations
errors_b=np.array(epsilonma(S_0, sigma_b))
Cst_b=np.array(simC(errors_b,rho_b,mu_b,cSource0_b))
Pst_b=np.array(simP(Cst_b,ModLoadProfile))


#Calculates simulated moments
for i in range (0, sim):

    sm1_b.append(np.mean(Pst_b[:,i]))
    sm2_b.append(np.var(Pst_b[:,i]*np.array(ModLoadProfile[:])))

    corrm3_b, pvalm3_b = sts.pearsonr(Pst_b[:,i],np.array(ModLoadProfile[:]))
    sm3_b.append(corrm3_b)

    for count in range (0, len(PSeries)-1):
        modPst_b[count,i]=Pst_b[count,i]
        modPst1_b[count,i]=Pst_b[count+1,i]

    corrm4_b, pvalm4_b = sts.pearsonr(modPst_b[:,i],modPst1_b[:,i])
    sm4_b.append(corrm4_b)
    sm5_b.append(np.sum(np.array(Pst_b[:,i])*np.array(ModLoadProfile[:])))


#Calculates errors (Difference between simulated moments and data moments)
err1_b=(np.mean(sm1_b)-m1)/m1
err2_b=(np.mean(sm2_b)-m2)/m2
err3_b=(np.mean(sm3_b)-m3)/m3
err4_b=(np.mean(sm4_b)-m4)/m4
err5_b=(np.mean(sm5_b)-m5)/m5

#Creates array of errors
err_b=np.array([err1_b, err2_b, err3_b, err4_b, err5_b])

#Prints final vector of moment differences
print("Vector of differences = ", err_b)

#Prints value of final estimator
print("Criterion function value =",results_uncstrb.fun)

#2. Generates Jacobian Matrix
npar=3
nmom=5
Ematrixb=np.zeros((nmom, npar))


for col in range (0,npar):
    derivparamup_b=np.array([rho_b, mu_b, sigma_b])
    derivparamup_b[col]=derivparamup_b[col]+h
    derivparamdown_b=np.array([rho_b, mu_b, sigma_b])
    derivparamdown_b[col]=derivparamdown_b[col]-h
    deltaerror_b=s_errors(derivparamup_b)-s_errors(derivparamdown_b)
    for row in range (0,nmom):
        Ematrixb[row,col]=deltaerror_b[row]/(2*h)
        
        
        
cov2b=(1/sim)*lin.pinv(Ematrixb.T@ W2 @ Ematrixb)

StandardErrb=[]

#3. Calculates and prints Standard errors.
for row in range (0,npar):
    StandardErrb.append((cov2b[row,row])**1/2)

print("Standard Errors of Parameter Vector = ", StandardErrb)

Estimated Rho =  0.6376260215823109
Estimated Mu =  1.217799715139845
Estimated Sigma =  3.5532190985103083
Vector of differences =  [ 0.14879016 -0.1323283  -0.70620266 -0.00110519  0.08211604]
Criterion function value = 0.9998626323760043
Standard Errors of Parameter Vector =  [2.225468317432432e-08, 2.2570585677098527e-06, 3.326202595177355e-07]


In [7]:
print(results_uncstr)
print("")
print(results_uncstrb)

rho, mu, sigma = results_uncstrb.x
print(rho, mu, sigma)

      fun: 0.5451157304682422
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([9.99200722e-08, 1.11022302e-08, 1.11022302e-08])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 52
      nit: 11
   status: 0
  success: True
        x: array([0.63765032, 1.21789317, 3.55344782])

      fun: 0.9998626323760043
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.49329479e-03, 3.86446430e-04, 8.39328607e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 40
      nit: 8
   status: 0
  success: True
        x: array([0.63762602, 1.21779972, 3.5532191 ])
0.6376260215823109 1.217799715139845 3.5532190985103083


In [37]:
Cst=Cst_b

Elasticities=[-0.1]
ElasticitiesExch=[-0.5]
PassThrough=[1]

for Passes in PassThrough:
    
    for ExchangeElasticity in ElasticitiesExch:
        
        ExchangeEla=ExchangeElasticity
        
        for elasticity in Elasticities:
            
            prodHydro=np.zeros((len(PSeries)))
            revHydro=np.zeros((len(PSeries)))
            prodHydroCO2=np.zeros((len(PSeries)))
            revHydroCO2=np.zeros((len(PSeries)))
            prodNuclear=np.zeros((len(PSeries)))
            revNuclear=np.zeros((len(PSeries)))
            prodNuclearCO2=np.zeros((len(PSeries)))
            revNuclearCO2=np.zeros((len(PSeries)))
            prodWind=np.zeros((len(PSeries)))
            revWind=np.zeros((len(PSeries)))
            prodWindCO2=np.zeros((len(PSeries)))
            revWindCO2=np.zeros((len(PSeries)))
            prodGas=np.zeros((len(PSeries)))
            revGas=np.zeros((len(PSeries)))
            prodGasCO2=np.zeros((len(PSeries)))
            revGasCO2=np.zeros((len(PSeries)))
            emissions0=np.zeros((len(PSeries)))
            emissionsCO2=np.zeros((len(PSeries)))
            consumersPay=np.zeros((len(PSeries)))
            consumersPayCO2=np.zeros((len(PSeries)))
            taxCollectedCO2=np.zeros((len(PSeries)))
            HrLoadCO2=np.zeros((len(PSeries)))
            HrExchangeCO2=np.zeros((len(PSeries)))
            
            prodGasnonpeak=np.zeros((len(PSeries)))
            revGasnonpeak=np.zeros((len(PSeries)))
            prodGaspeak=np.zeros((len(PSeries)))
            revGaspeak=np.zeros((len(PSeries)))
            prodGasCO2nonpeak=np.zeros((len(PSeries)))
            revGasCO2nonpeak=np.zeros((len(PSeries)))
            prodGasCO2peak=np.zeros((len(PSeries)))
            revGasCO2peak=np.zeros((len(PSeries)))

            DemandElasticity=elasticity
            Pass=Passes

            for row in range (0,len(PSeries)):

                CSourcet=np.mean(Cst[row,:])
                HrNGPrice=HrNaturalGasPrices[row]

                Marginal0=MarginalGen(ModLoadProfile[row])
                generation0=0
                for gen in range (0,Marginal0):
                    generation0=generation0+GenStack0['PMax'][gen]

                Pt=MarginalCost(Marginal0,HrNGPrice,CSourcet)
                consumersPay[row]=Pt*LoadProfile[row]

                errorLoop=1
                tolerance=1e-6
                TempCounter=0
                KonstantNY=ModLoadProfile[row]/(Pt**DemandElasticity)
                KonstantExt=Exchange[row]/(Pt**ExchangeEla)
                LoadCO2_0=ModLoadProfile[row]
                MarginalCO2_0=MarginalGenCO2(LoadCO2_0)
                PCO2t_0=MarginalCostCO2(MarginalCO2_0,HrNGPrice,CSourcet,Pass)

                while errorLoop>tolerance:
                    
                    LoadCO2=KonstantNY*(PCO2t_0**DemandElasticity)
                    HrExch=KonstantExt*((Pt-(PCO2t_0-Pt))**ExchangeEla)
                    
                    MarginalCO2_0=MarginalGenCO2(LoadCO2-(HrExch-Exchange[row]))
                    PCO2t=MarginalCostCO2(MarginalCO2_0,HrNGPrice,CSourcet,Pass)

                    if PCO2t<=1e-3:
                        errorLoop=0
                    else:
                        errorLoop=np.abs(PCO2t-PCO2t_0)
                        LoadCO2=KonstantNY*(PCO2t**DemandElasticity)
                        PCO2t_0=PCO2t
                        TempCounter=TempCounter+1
                        
                    if errorLoop<tolerance or TempCounter>15:
                        break

                HrExchangeCO2[row]=HrExch        
                MarginalCO2=MarginalGenCO2(LoadCO2-(HrExch-Exchange[row]))
                
                generationCO2=0
                for gen in range (0,MarginalCO2):
                    generationCO2=generationCO2+GenStackCO2['PMax'][gen]

                PCO2t=MarginalCostCO2(MarginalCO2,HrNGPrice,CSourcet,Pass)

                HrLoadCO2[row]=LoadCO2+Exchange[row]
                
                if ExchangeEla==-1e-15:
                    consumersPayCO2[row]=(PCO2t*(HrLoadCO2[row]-HrExchangeCO2[row]))+(Pt*HrExchangeCO2[row])
                else:
                    consumersPayCO2[row]=PCO2t*HrLoadCO2[row]


                gen=0
                while gen<=Marginal0:
                    if gen==Marginal0:
                        Generation=ModLoadProfile[row]-generation0
                    if gen<Marginal0:
                        Generation=GenStack0['PMax'][gen]

                    if GenStack0['FuelType'][gen]=="HYDRO":
                        prodHydro[row]=prodHydro[row]+Generation
                        revHydro[row]=revHydro[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="WIND":
                        prodWind[row]=prodWind[row]+Generation
                        revWind[row]=revWind[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="NUCLEAR":
                        prodNuclear[row]=prodNuclear[row]+Generation
                        revNuclear[row]=revNuclear[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="GAS":
                        prodGas[row]=prodGas[row]+Generation
                        revGas[row]=revGas[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        emissions0[row]=emissions0[row]+(Generation*GenStack0.EmissionRate.iloc[gen]*453.592/(1000000))
                        if gen<230:
                            prodGasnonpeak[row]=prodGasnonpeak[row]+Generation
                            revGasnonpeak[row]=revGasnonpeak[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        else:
                            prodGaspeak[row]=prodGaspeak[row]+Generation
                            revGaspeak[row]=revGaspeak[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        

                    gen=gen+1


                gen=0
                while gen<=MarginalCO2:
                    if gen==MarginalCO2:
                        Generation=HrLoadCO2[row]-HrExchangeCO2[row]-generationCO2
                    if gen<MarginalCO2:
                        Generation=GenStackCO2['PMax'][gen]

                    if GenStackCO2['FuelType'][gen]=="HYDRO":
                        prodHydroCO2[row]=prodHydroCO2[row]+Generation
                        revHydroCO2[row]=revHydroCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="WIND":
                        prodWindCO2[row]=prodWindCO2[row]+Generation
                        revWindCO2[row]=revWindCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="NUCLEAR":
                        prodNuclearCO2[row]=prodNuclearCO2[row]+Generation
                        revNuclearCO2[row]=revNuclearCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="GAS":
                        prodGasCO2[row]=prodGasCO2[row]+Generation
                        revGasCO2[row]=revGasCO2[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))
                        emissionsCO2[row]=emissionsCO2[row]+(Generation*GenStackCO2.EmissionRate.iloc[gen]*453.592/(1000000))
                        taxCollectedCO2[row]=taxCollectedCO2[row]+(Generation*GenStackCO2.EmissionRate.iloc[gen]*SCC*453.592/(1000000))
                        if gen<230:
                            prodGasCO2nonpeak[row]=prodGasCO2nonpeak[row]+Generation
                            revGasCO2nonpeak[row]=revGasCO2nonpeak[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))
                        else:
                            prodGasCO2peak[row]=prodGasCO2peak[row]+Generation
                            revGasCO2peak[row]=revGasCO2peak[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))


                    gen=gen+1

            hydroOut=np.sum(prodHydro)
            hydroProf=np.sum(revHydro)
            hydroOutCO2=np.sum(prodHydroCO2)
            hydroProfCO2=np.sum(revHydroCO2)
            WindOut=np.sum(prodWind)
            WindProf=np.sum(revWind)
            WindOutCO2=np.sum(prodWindCO2)
            WindProfCO2=np.sum(revWindCO2)
            NuclearOut=np.sum(prodNuclear)
            NuclearProf=np.sum(revNuclear)
            NuclearOutCO2=np.sum(prodNuclearCO2)
            NuclearProfCO2=np.sum(revNuclearCO2)
            GasOut=np.sum(prodGas)
            GasProf=np.sum(revGas)
            GasOutCO2=np.sum(prodGasCO2)
            GasProfCO2=np.sum(revGasCO2)
            emiss=np.sum(emissions0)
            emissCO2=np.sum(emissionsCO2)
            consumerExp=np.sum(consumersPay)
            consumerExpCO2=np.nansum(consumersPayCO2)
            taxCollCO2=np.sum(taxCollectedCO2)
            LoadCO2Scenario=np.sum(HrLoadCO2)
            GasOutnonpeak=np.sum(prodGasnonpeak)
            GasProfnonpeak=np.sum(revGasnonpeak)            
            GasOutpeak=np.sum(prodGaspeak)
            GasProfpeak=np.sum(revGaspeak)  
            GasOutCO2nonpeak=np.sum(prodGasCO2nonpeak)
            GasProfCO2nonpeak=np.sum(revGasCO2nonpeak)            
            GasOutCO2peak=np.sum(prodGasCO2peak)
            GasProfCO2peak=np.sum(revGasCO2peak)            
            
            print("Demand Elasticity =",DemandElasticity)
            print("Exchange Elasticity =", ExchangeEla)
            print("PassThrough=",Pass)
            print("")
            print(hydroOut)
            print(hydroProf)
            print(hydroOutCO2)
            print(hydroProfCO2)
            print(WindOut)
            print(WindProf)
            print(WindOutCO2)
            print(WindProfCO2)
            print(NuclearOut)
            print(NuclearProf)
            print(NuclearOutCO2)
            print(NuclearProfCO2)
            print(GasOut)
            print(GasProf)
            print(GasOutCO2)
            print(GasProfCO2)
            print(emiss)
            print(emissCO2)
            print(consumerExp)
            print(consumerExpCO2)
            print(taxCollCO2)
            print(LoadCO2Scenario)
            print("")
            print(GasOutnonpeak)
            print(GasProfnonpeak)
            print(GasOutpeak)
            print(GasProfpeak)
            print(GasOutCO2nonpeak)
            print(GasProfCO2nonpeak)
            print(GasOutCO2peak)
            print(GasProfCO2peak)
            print("")
            
        

Demand Elasticity = -0.1
Exchange Elasticity = -0.5
PassThrough= 1

52494928.218277305
2028979774.257631
52077273.63066705
2611464114.8053546
7702168.456970399
297678605.5431585
7702168.456970399
383136887.8021105
82334154.49584
3182105979.0916505
82334154.49584
4095632533.8231316
104587743.62099572
634839897.4820065
81446861.36090235
635612545.7140789
39764703.68177485
30223441.702805694
12533404388.476725
15889608388.786566
1088043901.301005
310126673.2610187

102709124.42203446
632027691.8853762
1878619.1989612565
2812205.5966303227
79297678.13275392
631113618.9468265
2149183.2281484418
4498926.767252428



In [13]:
TotCost=0
for line in range(0,len(PSeries)):
    TotCost=TotCost+(PSeries[line]*LoadProfile[line])

print(TotCost/np.sum(LoadProfile))
print(consumerExp/np.sum(LoadProfile))

35.928374351770074
39.47723736700671


In [14]:
print(np.mean(Cst))

1.2179119402937582


In [36]:
ExpLoadProfile=pd.DataFrame(LoadProfile)
ExpLoadProfile.columns=['MWh']
with open('LoadProfile.xlsx', "wb+") as f:
    
    ExpLoadProfile.to_excel(f, index=False)
    f.close
    
ExpPSeries=pd.DataFrame(PSeries)
ExpPSeries.columns=['MWh']
with open('PSeries.xlsx', "wb+") as f:
    ExpPSeries.to_excel(f, index=False)
    f.close

In [23]:
print((np.sum(HrLoadCO2)-np.sum(Exchange))/(1e6*2))
print(np.sum(ModLoadProfile)/(1e6*2))
print(np.average(HrNaturalGasPrices))

104.71651331569096
123.55949739604168
3.784095890410959


In [15]:
np.average(HrNaturalGasPrices)+np.mean(Cst)

5.002007830704717

In [40]:
with open('results_uncstr','wb') as f:
    pkl.dump(results_uncstr, f)
    
with open('results_uncstrb','wb') as f:
    pkl.dump(results_uncstrb, f)

In [41]:
with open('results_uncstr','rb') as f:
    results_uncstr = pkl.load(f)

with open('results_uncstrb','rb') as f:
    results_uncstrb = pkl.load(f)
    
rho, mu, sigma = results_uncstrb.x

#Prints estimated parameters
print("Estimated Rho = ", rho)
print("Estimated Mu = ", mu)
print("Estimated Sigma = ", sigma)

Estimated Rho =  0.6376503048702455
Estimated Mu =  1.217893209616846
Estimated Sigma =  3.5534478816297086


In [17]:
consumerExpCO2=np.nansum(consumersPayCO2)
print(consumerExpCO2)

10607265755.02834


In [20]:
np.sum(np.array(Exchange[:]*(np.array(consumersPay[:]/LoadProfile[:]))))

2734940430.436443

In [36]:
generation0=0
for gen in range (0,229):
    generation0=generation0+GenStack0['PMax'][gen]
print(generation0)

17846.423250049997


In [13]:
Cst=Cst_b

Elasticities=[-0.1]
ElasticitiesExch=[-0.5]
PassThrough=[1]

for Passes in PassThrough:
    
    for ExchangeElasticity in ElasticitiesExch:
        
        ExchangeEla=ExchangeElasticity
        
        for elasticity in Elasticities:
            
            prodHydro=np.zeros((len(PSeries)))
            revHydro=np.zeros((len(PSeries)))
            prodHydroCO2=np.zeros((len(PSeries)))
            revHydroCO2=np.zeros((len(PSeries)))
            prodNuclear=np.zeros((len(PSeries)))
            revNuclear=np.zeros((len(PSeries)))
            prodNuclearCO2=np.zeros((len(PSeries)))
            revNuclearCO2=np.zeros((len(PSeries)))
            prodWind=np.zeros((len(PSeries)))
            revWind=np.zeros((len(PSeries)))
            prodWindCO2=np.zeros((len(PSeries)))
            revWindCO2=np.zeros((len(PSeries)))
            prodGas=np.zeros((len(PSeries)))
            revGas=np.zeros((len(PSeries)))
            prodGasCO2=np.zeros((len(PSeries)))
            revGasCO2=np.zeros((len(PSeries)))
            emissions0=np.zeros((len(PSeries)))
            emissionsCO2=np.zeros((len(PSeries)))
            consumersPay=np.zeros((len(PSeries)))
            consumersPayCO2=np.zeros((len(PSeries)))
            taxCollectedCO2=np.zeros((len(PSeries)))
            HrLoadCO2=np.zeros((len(PSeries)))
            HrExchangeCO2=np.zeros((len(PSeries)))
            
            prodGasnonpeak=np.zeros((len(PSeries)))
            revGasnonpeak=np.zeros((len(PSeries)))
            prodGaspeak=np.zeros((len(PSeries)))
            revGaspeak=np.zeros((len(PSeries)))
            prodGasCO2nonpeak=np.zeros((len(PSeries)))
            revGasCO2nonpeak=np.zeros((len(PSeries)))
            prodGasCO2peak=np.zeros((len(PSeries)))
            revGasCO2peak=np.zeros((len(PSeries)))

            DemandElasticity=elasticity
            Pass=Passes

            for row in range (0, len(PSeries)):

                CSourcet=np.mean(Cst[row,:])
                HrNGPrice=HrNaturalGasPrices[row]

                Marginal0=MarginalGen(ModLoadProfile[row])
                generation0=0
                for gen in range (0,Marginal0):
                    generation0=generation0+GenStack0['PMax'][gen]

                Pt=MarginalCost(Marginal0,HrNGPrice,CSourcet)
                consumersPay[row]=Pt*LoadProfile[row]

                errorLoop=1
                tolerance=1e-6
                TempCounter=0
                KonstantNY=ModLoadProfile[row]/(Pt**DemandElasticity)
                KonstantExt=Exchange[row]/(Pt**ExchangeEla)
                LoadCO2_0=ModLoadProfile[row]
                MarginalCO2_0=MarginalGenCO2(LoadCO2_0)
                PCO2t_0=MarginalCostCO2(MarginalCO2_0,HrNGPrice,CSourcet,Pass)

                while errorLoop>tolerance:
                    
                    HrExch=KonstantExt*((Pt-(PCO2t_0-Pt))**ExchangeEla)
                    LoadC02=KonstantNY*(PCO2t_0**DemandElasticity)
                        
                    MarginalCO2=MarginalGenCO2(LoadCO2-(HrExch-Exchange[row]))
                    PCO2t=MarginalCostCO2(MarginalCO2,HrNGPrice,CSourcet,Pass)

                    if PCO2t<=1e-3:
                        errorLoop=0
                    else:
                        errorLoop=np.abs(PCO2t-PCO2t_0)

                        PCO2t_0=PCO2t
                        TempCounter=TempCounter+1
                        
                    if errorLoop<tolerance or TempCounter>15:
                        break

                HrExchangeCO2[row]=HrExch        
                
                generationCO2=0
                for gen in range (0,MarginalCO2):
                    generationCO2=generationCO2+GenStackCO2['PMax'][gen]

                HrLoadCO2[row]=LoadCO2+Exchange[row]
                
                if ExchangeEla==-1e-15:
                    consumersPayCO2[row]=(PCO2t*(HrLoadCO2[row]-HrExchangeCO2[row]))+(Pt*HrExchangeCO2[row])
                else:
                    consumersPayCO2[row]=PCO2t*HrLoadCO2[row]


                gen=0
                while gen<=Marginal0:
                    if gen==Marginal0:
                        Generation=ModLoadProfile[row]-generation0
                    if gen<Marginal0:
                        Generation=GenStack0['PMax'][gen]

                    if GenStack0['FuelType'][gen]=="HYDRO":
                        prodHydro[row]=prodHydro[row]+Generation
                        revHydro[row]=revHydro[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="WIND":
                        prodWind[row]=prodWind[row]+Generation
                        revWind[row]=revWind[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="NUCLEAR":
                        prodNuclear[row]=prodNuclear[row]+Generation
                        revNuclear[row]=revNuclear[row]+(Generation*Pt)
                    if GenStack0['FuelType'][gen]=="GAS":
                        prodGas[row]=prodGas[row]+Generation
                        revGas[row]=revGas[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        emissions0[row]=emissions0[row]+(Generation*GenStack0.EmissionRate.iloc[gen]*453.592/(1000000))
                        if gen<230:
                            prodGasnonpeak[row]=prodGasnonpeak[row]+Generation
                            revGasnonpeak[row]=revGasnonpeak[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        else:
                            prodGaspeak[row]=prodGaspeak[row]+Generation
                            revGaspeak[row]=revGaspeak[row]+(Generation*(Pt-MarginalCost(gen,HrNGPrice,CSourcet)))
                        

                    gen=gen+1


                gen=0
                while gen<=MarginalCO2:
                    if gen==MarginalCO2:
                        Generation=HrLoadCO2[row]-HrExchangeCO2[row]-generationCO2
                    if gen<MarginalCO2:
                        Generation=GenStackCO2['PMax'][gen]

                    if GenStackCO2['FuelType'][gen]=="HYDRO":
                        prodHydroCO2[row]=prodHydroCO2[row]+Generation
                        revHydroCO2[row]=revHydroCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="WIND":
                        prodWindCO2[row]=prodWindCO2[row]+Generation
                        revWindCO2[row]=revWindCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="NUCLEAR":
                        prodNuclearCO2[row]=prodNuclearCO2[row]+Generation
                        revNuclearCO2[row]=revNuclearCO2[row]+(Generation*PCO2t)
                    if GenStackCO2['FuelType'][gen]=="GAS":
                        prodGasCO2[row]=prodGasCO2[row]+Generation
                        revGasCO2[row]=revGasCO2[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))
                        emissionsCO2[row]=emissionsCO2[row]+(Generation*GenStackCO2.EmissionRate.iloc[gen]*453.592/(1000000))
                        taxCollectedCO2[row]=taxCollectedCO2[row]+(Generation*GenStackCO2.EmissionRate.iloc[gen]*SCC*453.592/(1000000))
                        if gen<230:
                            prodGasCO2nonpeak[row]=prodGasCO2nonpeak[row]+Generation
                            revGasCO2nonpeak[row]=revGasCO2nonpeak[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))
                        else:
                            prodGasCO2peak[row]=prodGasCO2peak[row]+Generation
                            revGasCO2peak[row]=revGasCO2peak[row]+(Generation*(PCO2t-MarginalCostCO2(gen,HrNGPrice,CSourcet,1)))


                    gen=gen+1

            hydroOut=np.sum(prodHydro)
            hydroProf=np.sum(revHydro)
            hydroOutCO2=np.sum(prodHydroCO2)
            hydroProfCO2=np.sum(revHydroCO2)
            WindOut=np.sum(prodWind)
            WindProf=np.sum(revWind)
            WindOutCO2=np.sum(prodWindCO2)
            WindProfCO2=np.sum(revWindCO2)
            NuclearOut=np.sum(prodNuclear)
            NuclearProf=np.sum(revNuclear)
            NuclearOutCO2=np.sum(prodNuclearCO2)
            NuclearProfCO2=np.sum(revNuclearCO2)
            GasOut=np.sum(prodGas)
            GasProf=np.sum(revGas)
            GasOutCO2=np.sum(prodGasCO2)
            GasProfCO2=np.sum(revGasCO2)
            emiss=np.sum(emissions0)
            emissCO2=np.sum(emissionsCO2)
            consumerExp=np.sum(consumersPay)
            consumerExpCO2=np.nansum(consumersPayCO2)
            taxCollCO2=np.sum(taxCollectedCO2)
            LoadCO2Scenario=np.sum(HrLoadCO2)
            GasOutnonpeak=np.sum(prodGasnonpeak)
            GasProfnonpeak=np.sum(revGasnonpeak)            
            GasOutpeak=np.sum(prodGaspeak)
            GasProfpeak=np.sum(revGaspeak)  
            GasOutCO2nonpeak=np.sum(prodGasCO2nonpeak)
            GasProfCO2nonpeak=np.sum(revGasCO2nonpeak)            
            GasOutCO2peak=np.sum(prodGasCO2peak)
            GasProfCO2peak=np.sum(revGasCO2peak)            
            
            print("")
            print("")
            print("Demand Elasticity =",DemandElasticity)
            print("Exchange Elasticity =", ExchangeEla)
            print("PassThrough=",Pass)
            print("")
            print(hydroOut)
            print(hydroProf)
            print(hydroOutCO2)
            print(hydroProfCO2)
            print(WindOut)
            print(WindProf)
            print(WindOutCO2)
            print(WindProfCO2)
            print(NuclearOut)
            print(NuclearProf)
            print(NuclearOutCO2)
            print(NuclearProfCO2)
            print(GasOut)
            print(GasProf)
            print(GasOutCO2)
            print(GasProfCO2)
            print(emiss)
            print(emissCO2)
            print(consumerExp)
            print(consumerExpCO2)
            print(taxCollCO2)
            print(LoadCO2Scenario)
            print("")
            print(GasOutnonpeak)
            print(GasProfnonpeak)
            print(GasOutpeak)
            print(GasProfpeak)
            print(GasOutCO2nonpeak)
            print(GasProfCO2nonpeak)
            print(GasOutCO2peak)
            print(GasProfCO2peak)
            print("")
            
        

/Users/Daniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in double_scalars




Demand Elasticity = -0.1
Exchange Elasticity = -0.5
PassThrough= 1

52494928.218277305
2028979774.257631
51765801.30194359
2020574877.0722027
7702168.456970399
297678605.5431585
7691617.541275919
296445494.152065
82334154.49584
3182105979.0916505
82220062.10310853
3168924342.731426
104587743.62099572
634839897.4820065
21498144.890328284
91359952.86864439
39764703.68177485
6814924.581991063
12533404388.476725
9430529101.729755
245337284.95167825
244719112.96715093

102709124.42203446
632027691.8853762
1878619.1989612565
2812205.5966303227
21498144.890328284
91359952.86864439
0.0
0.0

